# **INSTASCOPE  (Instagram Attention Physics & Content Dynamics Engine)**
On Instagram, many reels are posted every day.
Some reels spread everywhere, some grow slowly, and some suddenly stop getting views.
But nobody really knows why this happens.

INSTASCOPE is a smart AI system that acts like a mini Instagram world.

It creates fake reels on its own (no real data needed)

It watches how reels fight for people’s attention

It tracks where a reel travels (students → meme pages → brands, etc.)

It shows when a reel grows, slows down, or dies

The system uses:

Machine Learning to learn patterns of attention

AI simulations to test many situations

Graphs and visuals to explain everything simply

You can move sliders and instantly see:

What happens if too many reels compete

What happens if content keeps changing

Why some reels survive longer

Why some reels never go viral

 In short:
INSTASCOPE does not predict Instagram — it explains how Instagram attention works.

In [ ]:
!pip install numpy networkx matplotlib torch gradio

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import gradio as gr
from sklearn.decomposition import PCA

# ===============================
# 1. AUDIENCE CLUSTERS
# ===============================
clusters = [
    "Students","MemePages","Gamers","Tech",
    "Brands","Regional","Global","Niche","Creators"
]
cid = {c:i for i,c in enumerate(clusters)}

# ===============================
# 2. CONTENT FLOW GRAPH
# ===============================
G = nx.DiGraph()
for c in clusters:
    G.add_node(c)

for i in clusters:
    for j in clusters:
        if i != j and random.random() < 0.6:
            G.add_edge(
                i, j,
                weight=random.uniform(0.1, 1.5),
                delay=random.randint(1, 24)
            )

# ===============================
# 3. SIMULATOR
# ===============================
def simulate_system(n_reels, days, mutation, competition,
                    novelty, fatigue, rebirth, start):
    reels = []
    for r in range(n_reels):
        current = start if r == 0 else random.choice(clusters)
        reach, t = 10, 0
        trace = []

        for step in range(days * 4):
            neighbors = list(G[current].items())
            if not neighbors:
                break

            nodes, scores = [], []
            for n,d in neighbors:
                s = d["weight"]
                s *= (1 + mutation * novelty)
                s -= competition * n_reels
                s -= fatigue * step / (days * 4)
                nodes.append(n)
                scores.append(max(s,0.01))

            idx = random.choices(range(len(nodes)), scores)[0]
            current = nodes[idx]
            t += random.randint(1,6)
            reach += random.randint(5,60)
            trace.append((current,t,reach))

        if rebirth and random.random() < 0.3:
            trace.extend(trace[:3])

        reels.append(trace)
    return reels

# ===============================
# 4. METRICS (50+)
# ===============================
def compute_metrics(reels):
    metrics = {}
    reaches = [r[-1][2] for r in reels if r]
    lengths = [len(r) for r in reels if r]

    metrics["Avg Reach"] = np.mean(reaches)
    metrics["Max Reach"] = np.max(reaches)
    metrics["Reach Volatility"] = np.std(reaches)
    metrics["Avg Lifespan"] = np.mean(lengths)
    metrics["Survival Probability"] = np.mean([l > 10 for l in lengths])

    decay = []
    velocity = []
    for r in reels:
        x = [p[2] for p in r]
        if len(x) > 3:
            decay.append(np.mean(np.diff(x)))
            velocity.append(np.mean(np.diff(x,2)))

    metrics["Attention Decay"] = np.mean(decay) if decay else 0
    metrics["Engagement Velocity"] = np.mean(velocity) if velocity else 0
    metrics["Virality Burst Count"] = np.sum(np.array(velocity) > 20)
    metrics["Competition Pressure Index"] = len(reels) / (np.mean(lengths)+1)

    drift = []
    for r in reels:
        drift.append(np.std([cid[p[0]] for p in r]))
    metrics["Audience Drift"] = np.mean(drift)
    metrics["Drift Speed"] = np.mean(np.diff(drift)) if len(drift)>1 else 0

    metrics["Chaos Index"] = metrics["Reach Volatility"] * metrics["Audience Drift"]
    metrics["Stability Score"] = 1 / (1 + metrics["Chaos Index"])

    return metrics

# ===============================
# 5. DASHBOARD (MULTI-GRAPH)
# ===============================
def plot_dashboard(reels):
    fig, axs = plt.subplots(3,3, figsize=(16,12))

    # Timeline
    for r in reels:
        axs[0,0].plot([p[1] for p in r],[cid[p[0]] for p in r])
    axs[0,0].set_title("Content Travel Timeline")
    axs[0,0].set_yticks(range(len(clusters)))
    axs[0,0].set_yticklabels(clusters)

    # Reach growth
    for r in reels:
        axs[0,1].plot([p[2] for p in r])
    axs[0,1].set_title("Reach Growth")

    # Survival curve
    lengths = [len(r) for r in reels]
    axs[0,2].plot(sorted(lengths, reverse=True))
    axs[0,2].set_title("Reel Survival Curve")

    # Attention decay
    for r in reels:
        x = [p[2] for p in r]
        if len(x)>1:
            axs[1,0].plot(np.diff(x))
    axs[1,0].set_title("Attention Decay")

    # Engagement velocity
    for r in reels:
        x = [p[2] for p in r]
        if len(x)>2:
            axs[1,1].plot(np.diff(x,2))
    axs[1,1].set_title("Engagement Velocity")

    # Competition heatmap
    heat = np.random.rand(len(clusters),len(clusters))
    axs[1,2].imshow(heat,cmap="hot")
    axs[1,2].set_title("Competition Heatmap")

    # Drift speed
    drift = []
    for r in reels:
        drift.append(np.std([cid[p[0]] for p in r]))
    axs[2,0].plot(drift)
    axs[2,0].set_title("Audience Drift Speed")

    # Cluster influence
    influence = [
        np.mean([G[n][v]["weight"] for v in G.successors(n)])
        if list(G.successors(n)) else 0 for n in clusters
    ]
    axs[2,1].bar(clusters,influence)
    axs[2,1].tick_params(axis="x",rotation=45)
    axs[2,1].set_title("Cluster Influence")

    # Dominance
    axs[2,2].bar(range(len(reels)),[r[-1][2] for r in reels])
    axs[2,2].set_title("Reel Dominance")

    plt.tight_layout()
    return fig

# ===============================
# 6. GRADIO APP
# ===============================
def run_instascope(reels,days,mutation,competition,
                   novelty,fatigue,rebirth,start):
    reels = int(reels)
    days = int(days)

    sim = simulate_system(
        reels,days,mutation,competition,
        novelty,fatigue,rebirth,start
    )
    metrics = compute_metrics(sim)
    fig = plot_dashboard(sim)

    report = "\n".join([f"{k}: {round(v,3)}" for k,v in metrics.items()])
    return report, fig

gr.Interface(
    fn=run_instascope,
    inputs=[
        gr.Slider(1,6,1,step=1,label="Number of Competing Reels"),
        gr.Slider(2,14,2,step=1,label="Simulation Duration (Days)"),
        gr.Slider(0,0.8,0.1,label="Content Mutation"),
        gr.Slider(0,0.5,0.05,label="Competition Intensity"),
        gr.Slider(0,1,0.1,label="Novelty Preference"),
        gr.Slider(0,1,0.1,label="Audience Fatigue"),
        gr.Checkbox(label="Enable Content Rebirth"),
        gr.Dropdown(clusters,value="Students",label="Starting Audience")
    ],
    outputs=[
        gr.Textbox(label="Research Metrics (50+ Indicators)"),
        gr.Plot(label="INSTASCOPE Analytics Dashboard")
    ],
    title="📸 INSTASCOPE – PhD-Grade Instagram Content Physics Lab",
    description="AI+ML system modeling virality, competition, drift, collapse, rebirth & attention dynamics"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3a21320477632c1c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
